In [1]:
import json
import re
from pathlib import Path
import subprocess
import os

In [2]:
def merge_notebooks(folder_path="./", output_file="script.ipynb"):
    folder = Path(folder_path)
    notebooks = sorted(
        (nb for nb in folder.glob("*.ipynb") if re.match(r"^\d+\.", nb.name)),
        key=lambda p: int(re.match(r"(\d+)\.", p.name).group(1))
    )
    if not notebooks:
        return

    cells = []
    metadata = None

    for path in notebooks:
        nb = json.loads(Path(path).read_text(encoding="utf-8"))
        metadata = metadata or nb.get("metadata", {})
        cells.extend(nb.get("cells", []))

    merged = {
        "cells": cells,
        "metadata": metadata,
        "nbformat": 4,
        "nbformat_minor": 5
    }

    Path(output_file).write_text(json.dumps(merged, indent=2), encoding="utf-8")
    subprocess.run(["jupyter", "nbconvert", "--to", "html", output_file])
    os.remove(output_file)

merge_notebooks()